# Lecture 3 - Discretisation of the Radon Transform

# Contents

* The discrete Radon transform
* Sparse matrices
* Matrix-free implementation
* Assignments

# The Discrete Radon Transform


The Radon transform is defined as 

$$f(s,\theta) = \int_{-\infty}^\infty u\left(x(s,t), y(s,t)\right) \mathrm{d}t,$$

with

$$x(s,t) = s \cos \theta + t \sin \theta, \quad y(s,t) = s \sin \theta - t\cos\theta.$$

* Let $s \in [-1,1]$ and discretise the detector in $n_d$ cells with width $2/n_d$.
* Let $(x,y) \in [-1,1]^2$ and discretise the image in $n_x \times n_y$ pixels with size $2/n_x \times 2/n_y$.
* The contribution of the $j-th$ pixel to the $i-th$ detector element at angle $\theta$ is then given by 

$$a_{ij}(\theta) = ....$$

* The measurement along a single angle then leads to a matrix $A(\theta) \in \mathbb{R}^{n_d \times n_x \times n_y}$

We can now easily compute the matrix elements for a given set of angles $\{\theta_k\}_{k=1}^{n_\theta}$ and define

$$A = \left( \begin{matrix} A(\theta_1) \\ A(\theta_2) \\ \vdots \\ A(\theta_k)\end{matrix}\right).$$

The vectorised discretised sinogram and image are then related via

$$\mathbf{f} = A\mathbf{u}.$$

# Sparse matrices

# Matrix-free implementation

# Assignments